## Description

Typical office material supply company running their operations sees many challenges, one of the major ones being how to measure the effectiveness of sales and marketing operations. We have picked up an imaginary office material supply company **“Office Corp.”** for our analysis. They have tested a telemarketing campaign targeting existing domestic business customers with a random selection of products. For this exercise, a sample of 16,172 customers were targeted. The products being marketed are Desk, Executive Chair, Standard Chair, Monitor, Printer Computer, Insurance, Toner and Office Supplies.  

Office Corp would like to leverage the knowledge gained from this survey (and associated dataset) for future campaigns. Office Corp has hired an analyst team to solve the following three problems using various analytical techniques.  

- Profile the customers that responded to the campaign to understand the characteristics of customers who made purchases. 

- Develop models using the campaign results to target responsive, profitable customers for future campaigns. 

- Summarize and categorize the profitability of the various marketing segments which may be used to optimize future campaigns.

---

## Scientific questions/investigation

Given marketing dataset, there are three key problems to solve.  

- Given campaign period data, looking at customer profile, we aim to predict whether customer will Buy or Not Buy. We intent to frame this as a Classification problem. 

- Among the customers who are predicted as they will buy, we aim to estimate what will be size of purchase. We intent to frame this as a Regression problem. 

- For those customers who are likely to purchase and in general other customers using their historical product purchase data, we aim to predict top 3 products that are likely to be purchased. Also try to predict whether there are any product bundles Office Corp may sell more, etc. We intend to frame this as a Clustering problem.  

In the process of analysis, we intent to carry out,  

- data missing and imputation treatments,  

- data sufficiency and randomized splitting for test-train sets,  

- test amount of data necessary for optimal predictions & accuracy, etc.  

This last topic of determining optimal amount of data is called the ‘sample complexity study’ and it is extremely important for such real-life marketing campaign problems because collecting too much data unnecessarily may cause the company cost overrun. Data scientists should be able to advise the business department on how much data to collect to achieve a decent model performance. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Loading the data

In [ ]:
supply = pd.read_csv("office_supply.csv")

In [ ]:
supply = supply[['Customer Number', 'Service Level',' Date of Last Transaction ','Number of Transactions','Email Available']]
supply.columns = ['Customer Number', 'Service Level','Date of Last Transaction','Number of Transactions','Email Available']

In [ ]:
campaign = pd.read_excel("office_supply_campaign_results.xlsx", 
                         sheet_name="Campaign Results",
                         na_values=[' ',''], true_values=['TRUE'],false_values=['FALSE'])

### Columns

In [ ]:
campaign.columns

In [ ]:
for c1 in supply.columns:
    if c1 not in campaign.columns:
        print(c1)

### Join

In [ ]:
supply['Customer Number'] = supply['Customer Number'].apply(lambda x: int(x))

In [ ]:
campaign=campaign.dropna(axis=0,subset=['Customer Number'])

In [ ]:
campaign['Customer Number'] = campaign['Customer Number'].apply(lambda x: int(x))

In [ ]:
df = campaign.merge(supply,on='Customer Number')

In [ ]:
df.head()

In [ ]:
df['Date of Last Transaction'] = pd.to_datetime(df['Date of Last Transaction'])

In [ ]:
df['Number of Prior Year Transactions'] = df['Number of Prior Year Transactions'].apply(lambda x: int(x))

In [ ]:
df.info()

### Fill `Language` missing values by `Unknown` 

In [ ]:
df['Language'].fillna('Unknown',inplace=True)

### Dropping all other missing values

In [ ]:
df = df.dropna()

In [ ]:
df.columns

In [ ]:
df.info()

### Categorical variables

In [ ]:
cat_vars = ['Do Not Direct Mail Solicit', 'Do Not Email', 'Do Not Telemarket',
       'Repurchase Method', 'Last Transaction Channel', 'Desk',
       'Executive Chair', 'Standard Chair', 'Monitor', 'Printer', 'Computer',
       'Insurance', 'Toner', 'Office Supplies', 'Number of Employees', 'Language','Service Level','Email Available']

In [ ]:
len(cat_vars)

### What are the unique values in the columns?

In [ ]:
for v in cat_vars:
    print(f"Values in {v}: {df[v].unique()}")

### Converting boolean columns to proper data types

In [ ]:
#type_dict = {}


In [ ]:
df=df.astype({'Do Not Direct Mail Solicit': 'bool', 'Do Not Email':bool, 'Do Not Telemarket':bool})

In [ ]:
df.info()

In [ ]:
fig, ax = plt.subplots(6,3,figsize=(18,25),squeeze=False,sharey=True)
fig.tight_layout(pad=2.0)
ax = ax.ravel()
for i in range(18):
    sns.stripplot(x=cat_vars[i],y='Campaign Period Sales',data=df,ax=ax[i],edgecolor='k')
    ax[i].set_xlabel(cat_vars[i],fontsize=14)
    if len(df[cat_vars[i]].unique()) > 3:
        ax[i].set_xticklabels(labels=df[cat_vars[i]].unique(), rotation = 90)
plt.show()

In [ ]:
fig, ax = plt.subplots(6,3,figsize=(12,18),squeeze=True,sharey=True)
ax = ax.ravel()
for i in range(18):
    ax[i].pie(df[cat_vars[i]].value_counts(),
        labels=df[cat_vars[i]].unique())
    ax[i].set_xlabel(cat_vars[i],fontsize=14)
plt.show()

### Buy/No buy?

In [ ]:
df['Buy'] = (df['Campaign Period Sales'] > 0).apply(lambda x: int(x))

In [ ]:
plt.hist(df['Buy'])
plt.show()

### Customer engagement length (days)

In [ ]:
df['Customer_engagement_days'] = (pd.Timestamp.now().normalize() - df['Date of First Purchase']).dt.days

In [ ]:
df['Days_since_last_transaction'] = (pd.Timestamp.now().normalize() - df['Date of Last Transaction']).dt.days

In [ ]:
plt.hist(df['Customer_engagement_days']/365,bins=25,edgecolor='k')
plt.show()

In [ ]:
df['Date of First Purchase'].describe()

In [ ]:
plt.hist(df['Days_since_last_transaction'],bins=25,edgecolor='k')
plt.show()

In [ ]:
def customer_engagement(d):
    m = df['Customer_engagement_days'].mean()
    s = df['Customer_engagement_days'].std()
    if d >= m+s:
        return 'Long-term'
    elif d > m-s and d < m+s:
        return 'Mid-term'
    else:
        return 'Short-term'

In [ ]:
m = df['Customer_engagement_days'].mean()
s = df['Customer_engagement_days'].std()

print(m/365,s/365)

In [ ]:
df['Customer_engagement_length'] = df['Customer_engagement_days'].apply(customer_engagement)

In [ ]:
sns.stripplot(x='Customer_engagement_length',y='Campaign Period Sales',data=df)
plt.show()

In [ ]:
df2 = df[df['Buy']==1]
plt.scatter(df2['Customer_engagement_days']/365,df2['Campaign Period Sales'])
plt.show()

### Language grouping

In [ ]:
df['Language'].unique()

In [ ]:
def language_group(lan):
    if lan=='English':
        return 'English'
    elif lan=='Hindi' or lan=='Chinese' or lan=='Hebrew' or lan=='Japanese' or lan=='Arabic' or lan=='Vietnamese' or lan=='Thai' or lan=='Pashto':
        return 'Asian'
    elif lan=='Unknown':
        return 'Unknown'
    else:
        return 'European'        

In [ ]:
df['Language_group'] = df['Language'].apply(language_group)

In [ ]:
df.info()

In [ ]:
plt.hist(df[df['Buy']==1]['Campaign Period Sales'],bins=25,edgecolor='k')
plt.xlim(0,2000)
plt.show()

### Saving to a CSV

In [ ]:
df.to_csv('Cleaned_campaign_sale.csv')

In [ ]:
d=pd.get_dummies(df,columns=cat_vars)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
d.info()

In [ ]:
d.head()

In [ ]:
len(d[d['Campaign Period Sales']==0.0])

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(18,6))
ax[0].scatter(d['Historical Sales Volume'],d['Campaign Period Sales'],c='blue',edgecolor='k')
ax[0].set_title('Historical Sales Volume',fontsize=18)
ax[1].scatter(d['Number of Prior Year Transactions'],d['Campaign Period Sales'],c='blue',edgecolor='k')
ax[1].set_title('Number of Prior Year Transactions',fontsize=18)
plt.show()